<a href="https://colab.research.google.com/github/sronast/NN/blob/master/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  !pip install -q tf-nightly
except Exception:
  pass
import tensorflow as tf


     |████████████████████████████████| 467.6MB 34kB/s 
     |████████████████████████████████| 460kB 51.2MB/s 
     |████████████████████████████████| 3.9MB 50.1MB/s 
     |████████████████████████████████| 2.9MB 49.4MB/s 
     |████████████████████████████████| 81kB 9.2MB/s 
ERROR: tensorflow 1.15.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
ERROR: tb-nightly 2.2.0a20200212 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.


In [2]:
from tensorflow import keras

!pip install -q tensorflow-datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

import numpy as np

print(tf.__version__)

2.2.0-dev20200212


In [3]:
(train_data, test_data), info = tfds.load('imdb_reviews/subwords8k', split=(tfds.Split.TRAIN, tfds.Split.TEST), as_supervised=True, with_info=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteA0UG5N/imdb_reviews-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteA0UG5N/imdb_reviews-test.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteA0UG5N/imdb_reviews-unsupervised.tfrecord
Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [0]:
encoder = info.features['text'].encoder

In [5]:
print ('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


In [6]:
print('Tensorflow is encoded as: {}'.format(encoder.encode('Tensorflow')))

Tensorflow is encoded as: [6307, 2327, 2934]


In [7]:
type(train_data)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [9]:
for train_example, train_label in train_data.take(1):
  print('Encoded text:', train_example[:10].numpy())
  print('Label:', train_label.numpy())

Encoded text: [  62   18   41  604  927   65    3  644 7968   21]
Label: 0


In [10]:
encoder.decode(train_example)

"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

In [0]:
BUFFER_SIZE = 1000

train_batches = (
    train_data
    .shuffle(BUFFER_SIZE)
    .padded_batch(32))

test_batches = (
    test_data
    .padded_batch(32))

In [14]:
for example_batch, label_batch in train_batches.take(2):
  print("Batch shape:", example_batch.shape)
  print("label shape:", label_batch.shape)
  

Batch shape: (32, 770)
label shape: (32,)
Batch shape: (32, 902)
label shape: (32,)


In [15]:
model = keras.Sequential([
  keras.layers.Embedding(encoder.vocab_size, 16),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(1)])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 130,977
Trainable params: 130,977
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [17]:
history = model.fit(train_batches,
                    epochs=10,
                    validation_data=test_batches,
                    validation_steps=30)

Epoch 1/10
782/782 [==============================] - 13s 16ms/step - loss: 0.6825 - accuracy: 0.5004 - val_loss: 0.6677 - val_accuracy: 0.5083
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.6240 - accuracy: 0.5492 - val_loss: 0.5984 - val_accuracy: 0.6094
Epoch 3/10
782/782 [==============================] - 9s 12ms/step - loss: 0.5452 - accuracy: 0.6573 - val_loss: 0.5360 - val_accuracy: 0.7146
Epoch 4/10
782/782 [==============================] - 10s 12ms/step - loss: 0.4783 - accuracy: 0.7464 - val_loss: 0.4849 - val_accuracy: 0.7802
Epoch 5/10
782/782 [==============================] - 11s 14ms/step - loss: 0.4257 - accuracy: 0.8006 - val_loss: 0.4475 - val_accuracy: 0.7812
Epoch 6/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3826 - accuracy: 0.8316 - val_loss: 0.4194 - val_accuracy: 0.8000
Epoch 7/10
782/782 [==============================] - 9s 12ms/step - loss: 0.3516 - accuracy: 0.8512 - val_loss: 0.3975 - val_accuracy: 0

In [18]:
loss, accuracy = model.evaluate(test_batches)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

    782/Unknown - 6s 8ms/step - loss: 0.3330 - accuracy: 0.8544Loss:  0.33297601039223657
Accuracy:  0.85444
